@inoichan
https://www.guruguru.science/competitions/25/discussions/a85b8a5a-2041-4a1b-84fc-5ad5492c3978/


In [1]:
import os
import sys
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import pipeline

# ==================================
# Dataの前処理に関するConfig
# ==================================
PATH_TO_DATASET = "../"


def get_relative_path(path):
    return os.path.join(PATH_TO_DATASET, path)


# 画像へのパス
image_path_root_list = [
    get_relative_path("atmaCup#18_dataset/images/{ID}/image_t.png"),
    get_relative_path("atmaCup#18_dataset/images/{ID}/image_t-0.5.png"),
    get_relative_path("atmaCup#18_dataset/images/{ID}/image_t-1.0.png"),
]

# 特徴量のパス
train_feature_path = get_relative_path("atmaCup#18_dataset/train_features.csv")
traffic_light_path = get_relative_path("atmaCup#18_dataset/traffic_lights/{ID}.json")

# 信号機の情報へのパス
test_feature_path = get_relative_path("atmaCup#18_dataset/test_features.csv")


# ========================================
# DataFrameの読み込み
# ========================================
df_feature_train = pd.read_csv(train_feature_path)
df_feature_test = pd.read_csv(test_feature_path)

# =======================================
# 画像のパスの追加
# =======================================
df_feature_train["img_path_t_00"] = [image_path_root_list[0].format(ID=ID) for ID in df_feature_train.ID]
df_feature_train["img_path_t_05"] = [image_path_root_list[1].format(ID=ID) for ID in df_feature_train.ID]
df_feature_train["img_path_t_10"] = [image_path_root_list[2].format(ID=ID) for ID in df_feature_train.ID]

df_feature_test["img_path_t_00"] = [image_path_root_list[0].format(ID=ID) for ID in df_feature_test.ID]
df_feature_test["img_path_t_05"] = [image_path_root_list[1].format(ID=ID) for ID in df_feature_test.ID]
df_feature_test["img_path_t_10"] = [image_path_root_list[2].format(ID=ID) for ID in df_feature_test.ID]

df_feature = pd.concat([df_feature_train, df_feature_test], axis=0, ignore_index=True)

# =======================================
# Depth Mapの生成と保存
# =======================================
depth_anything_v2 = pipeline(task="depth-estimation", model="depth-anything/Depth-Anything-V2-Small-hf", device=0)

for i in tqdm(range(len(df_feature))):
    row = df_feature.iloc[i]

    for t, image_path_root in enumerate(image_path_root_list):
        img_pil = Image.open(image_path_root.format(ID=row.ID))
        pred = depth_anything_v2(img_pil)

        depth_path = f"depth/{row.ID}/{t}.png"
        os.makedirs(os.path.dirname(depth_path), exist_ok=True)
        pred["depth"].save(depth_path)

2024-11-18 11:14:39.917755: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-18 11:14:39.941980: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:230: UserWarning: 
NVIDIA GeForce RTX 4090 with CUDA capability sm_89 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_60 sm_70 sm_75 compute_70 compute_75.
If you want to use the NVIDIA GeForce RTX 4090 GPU with PyTorch, please check the instructions at h